In [19]:
import mysql.connector
import os
import requests as r
import json
from pandas import DataFrame
from datetime import date
import numpy as np

import holidays

def lookup():
    val_list = []
    validator = open("validation.txt", "r")
    for line in validator:
        val_list.append(line.rstrip())

    mydb = mysql.connector.connect(
      host=val_list[0],
      user=val_list[1],
      passwd=val_list[2],
      database=val_list[3],
      port=val_list[4]
    )

    mycursor = mydb.cursor()
    result_list = []

    sql = """SELECT CONCAT(str_to_date(leavetimes.`dayofservice`,'%Y-%m-%d')," ",SEC_TO_TIME(leavetimes.`actualtime_arr`)) as date,leavetimes.tripid,leavetimes.progrnumber,leavetimes.stoppointid,leavetimes.actualtime_arr, weather.precipIntensity,weather.temperature,trips.lineid, stops.lat, stops.lng FROM leavetimes, weather, trips, stops WHERE weather.time = DATE_FORMAT((CONCAT(str_to_date(leavetimes.`dayofservice`,'%Y-%m-%d')," ",SEC_TO_TIME(leavetimes.`actualtime_arr`))), "%Y-%m-%d %H:00:00") and trips.tripid = leavetimes.tripid and trips.dayofservice = leavetimes.dayofservice and leavetimes.stoppointid = stops.stop_id ORDER BY leavetimes.dayofservice,leavetimes.tripid,leavetimes.progrnumber LIMIT 0,20"""
    mycursor.execute(sql)

    for (i) in mycursor:
        result_list.extend([i])
    frame = DataFrame.from_records(result_list)
    frame.columns = ["Date-Time","Trip_ID","Prog_No","Stop_ID","Time","Precip","Heat","lineid"]
    return frame

DFRAME = lookup()
DFRAME




ValueError: Length mismatch: Expected axis has 10 elements, new values have 8 elements

In [23]:
import sqlite3
import pandas as pd

# Open the file
f = open('output.csv', 'w')
# Create a connection and get a cursor
val_list = []
validator = open("validation.txt", "r")
for line in validator:
    val_list.append(line.rstrip())

mydb = mysql.connector.connect(
      host=val_list[0],
      user=val_list[1],
      passwd=val_list[2],
      database=val_list[3],
      port=val_list[4]
    )

mycursor = mydb.cursor()
result_list = []
# Execute the query
sql = """SELECT CONCAT(str_to_date(leavetimes.`dayofservice`,'%Y-%m-%d')," ",SEC_TO_TIME(leavetimes.`actualtime_arr`)) as date,leavetimes.tripid,leavetimes.progrnumber,leavetimes.stoppointid,leavetimes.actualtime_arr, weather.precipIntensity,weather.temperature,trips.lineid, stops.lat, stops.lng FROM leavetimes, weather, trips, stops WHERE weather.time = DATE_FORMAT((CONCAT(str_to_date(leavetimes.`dayofservice`,'%Y-%m-%d')," ",SEC_TO_TIME(leavetimes.`actualtime_arr`))), "%Y-%m-%d %H:00:00") and trips.tripid = leavetimes.tripid and trips.dayofservice = leavetimes.dayofservice and leavetimes.stoppointid = stops.stop_id ORDER BY leavetimes.dayofservice,leavetimes.tripid,leavetimes.progrnumber"""
mycursor.execute(sql)


# Get data in batches
while True:
    # Read the data
    df = pd.DataFrame(mycursor.fetchmany(1000))
    # We are done if there are no data
    if len(df) == 0:
        break
    # Let's write to the file
    else:
        df.to_csv(f, header=False)

# Clean up
f.close()
cursor.close()
connection.close()

KeyboardInterrupt: 